# Testing the `BranchedCoverHomology` class

## Setup

In [1]:
import notebook_setup

Notebook setup complete. Environment configured.


In [2]:
%preparse gaknot
from gaknot.gaknot import GeneralizedAlgebraicKnot

INFO: 

import_sage called with arguments:
	module_name: gaknot
	package: gaknot
	path: /Users/wojtek/Library/CloudStorage/GoogleDrive-w.politarczyk@uw.edu.pl/My Drive/mat/git-projects/signature_function
Successfully preparsed and reloaded: gaknot


In [3]:
%preparse H1_branched_cover
from gaknot.H1_branched_cover import BranchedCoverHomology

INFO: 

import_sage called with arguments:
	module_name: H1_branched_cover
	package: gaknot
	path: /Users/wojtek/Library/CloudStorage/GoogleDrive-w.politarczyk@uw.edu.pl/My Drive/mat/git-projects/signature_function
Successfully preparsed and reloaded: H1_branched_cover


In [4]:
from sage.all import ZZ

## Tests

In [8]:
# Test 1: Trefoil T(2,3) at N=2
# Expected: A single layer with H_1 = Z/3Z
knot_trefoil = GeneralizedAlgebraicKnot([(1, [(2, 3)])])
h1_trefoil = BranchedCoverHomology(knot_trefoil, 2)

print("--- Test 1: Trefoil T(2,3) at N=2 ---")
# Access the first component of the connected sum
component = h1_trefoil[0] 
layers = component['layers']

print(f"Number of layers: {len(layers)}")
print(f"Homology group: {h1_trefoil}")
print(f"Layer 0 details: {layers[0]}")

# Assertions
assert len(h1_trefoil) == 1
assert len(layers) == 1
assert layers[0]['base_factors'] == [3]
assert layers[0]['multiplicity'] == 1
assert layers[0]['effective_N'] == 2

print("PASSED: Basic structure verified.\n")

--- Test 1: Trefoil T(2,3) at N=2 ---
Number of layers: 1
Homology group: (Z/3Z)[T(2,3)]
Layer 0 details: {'cable_index': 0, 'parameters': (2, 3), 'effective_N': 2, 'multiplicity': 1, 'base_factors': [3]}
PASSED: Basic structure verified.



In [10]:
# Test 2: Iterated Knot T(2,3; 2,5) at N=2
knot_iter = GeneralizedAlgebraicKnot([(1, [(2, 3), (2, 5)])])
h1_iter = BranchedCoverHomology(knot_iter, 2)

print("--- Test 2: Iterated T(2,3; 2,5) at N=2 ---")
component = h1_iter[0]
layers = component['layers']

print(f"Total Invariant Factors (Flattened): {h1_iter.invariant_factors}")
print(f"Homology group: {h1_iter}")

# Layer 0 (Outermost, T(2,5))
layer_outer = layers[0]
print(f"Outer Layer (Index 1): {layer_outer}")
assert layer_outer['cable_index'] == 1
assert layer_outer['parameters'] == (2, 5)
assert layer_outer['effective_N'] == 2
assert layer_outer['base_factors'] == [5]

# Layer 1 (Innermost, T(2,3))
layer_inner = layers[1]
print(f"Inner Layer (Index 0): {layer_inner}")
assert layer_inner['cable_index'] == 0
assert layer_inner['parameters'] == (2, 3)
assert layer_inner['effective_N'] == 1
assert layer_inner['multiplicity'] == 2 # gcd(2,2)
assert layer_inner['base_factors'] == [] # Homology of 1-fold cover is trivial

print("PASSED: Satellite recursion logic verified.\n")

--- Test 2: Iterated T(2,3; 2,5) at N=2 ---
Total Invariant Factors (Flattened): [5]
Homology group: (Z/5Z)[T(2,3; 2,5)]
Outer Layer (Index 1): {'cable_index': 1, 'parameters': (2, 5), 'effective_N': 2, 'multiplicity': 1, 'base_factors': [5]}
Inner Layer (Index 0): {'cable_index': 0, 'parameters': (2, 3), 'effective_N': 1, 'multiplicity': 2, 'base_factors': []}
PASSED: Satellite recursion logic verified.



In [11]:
# Test 3: Connected Sum T(2,3) # -T(2,3; 2,5) at N=2
knot_t23 = GeneralizedAlgebraicKnot([(1, [(2, 3)])])
knot_iter_neg = GeneralizedAlgebraicKnot([(-1, [(2, 3), (2, 5)])])

knot_sum = knot_t23 + knot_iter_neg
h1_sum = BranchedCoverHomology(knot_sum, 2)

print("--- Test 3: Connected Sum Decomposition ---")
print(f"Full String: {h1_sum}")

# Check Component 0 (Trefoil)
comp0 = h1_sum[0]
print(f"Component 0 Layers: {len(comp0['layers'])}")
assert comp0['sign'] == 1
assert comp0['layers'][0]['base_factors'] == [3]

# Check Component 1 (Iterated Negative)
comp1 = h1_sum[1]
print(f"Component 1 Layers: {len(comp1['layers'])}")
assert comp1['sign'] == -1
# Verify the inner layer logic persists in the second component
assert comp1['layers'][1]['effective_N'] == 1 
assert comp1['layers'][1]['multiplicity'] == 2

print("PASSED: Connected sum structure verified.\n")

--- Test 3: Connected Sum Decomposition ---
Full String: (Z/3Z)[T(2,3)] ⊕ (Z/5Z)[-T(2,3; 2,5)]
Component 0 Layers: 1
Component 1 Layers: 2
PASSED: Connected sum structure verified.



In [12]:
# Test 4: Homology Object Addition
h1_part1 = BranchedCoverHomology(knot_t23, 2)
h1_part2 = BranchedCoverHomology(knot_iter_neg, 2)

h1_combined = h1_part1 + h1_part2

print("--- Test 4: Homology Addition ---")
print(f"Combined String: {h1_combined}")

# Verify length
assert len(h1_combined) == 2

# Verify indices are updated correctly
assert h1_combined[0]['index'] == 0
assert h1_combined[1]['index'] == 1

# Verify structure preservation
assert len(h1_combined[1]['layers']) == 2

print("PASSED: Addition logic verified.\n")

--- Test 4: Homology Addition ---
Combined String: (Z/3Z)[T(2,3)] ⊕ (Z/5Z)[-T(2,3; 2,5)]
PASSED: Addition logic verified.

